# Nhận diện cảm xúc từ âm thanh
Sử dụng Logistic Regression

## I. Import thư viện cần thiết

In [ ]:
# Import necessary libraries
# pandas and numpy for data manipulation and numerical operations
import pandas as pd
import numpy as np

# os and sys for file and system path handling
import os
import sys

# librosa is a library for audio processing, used here to extract features from audio files
import librosa
import librosa.display

# seaborn and matplotlib for data visualization (plots)
import matplotlib.pyplot as plt

# sklearn libraries for preprocessing and splitting data
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# sklearn libraries for generating confusion matrix and detailed classification report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Seaborn library for visualization of the confusion matrix
import seaborn as sns

# IPython's Audio class for playing audio files directly in Jupyter
from IPython.display import Audio

# Suppress warnings to avoid cluttering the output
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

## II. Chuẩn bị dữ liệu - Data Preparation
Khai báo đường dẫn đến các tập dữ liệu

In [ ]:
# Define dataset paths

# RAVDESS: Ryerson Audio-Visual Database of Emotional Speech and Song
Ravdess = "data/ravdess-emotional-speech-audio/audio_speech_actors_01-24/archive/"

# CREMA-D: Crowd-sourced Emotional Multimodal Actors Dataset
Crema = "data/cremad/AudioWAV/"

# TESS: Toronto Emotional Speech Set
Tess = "data/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"

# SAVEE: Surrey Audio-Visual Expressed Emotion
Savee = "data/surrey-audiovisual-expressed-emotion-savee/ALL/"

###     1. RAVDESS

In [ ]:
# Get the list of subdirectories in the RAVDESS dataset directory
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []  # List to store the emotion label for each audio file
file_path = []  # List to store the full path of each audio file

for dir in ravdess_directory_list:
    if dir == '.DS_Store':  # Skip MacOS system file
        continue
    # Each subdirectory corresponds to an actor and contains audio files
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]  # Remove the file extension (.wav)
        part = part.split('-')  # Split the filename into parts based on RAVDESS naming convention

        # The third element in the filename represents the emotion
        file_emotion.append(int(part[2]))

        # Construct and save the full path to the audio file
        file_path.append(Ravdess + dir + '/' + file)

# Create a DataFrame for the extracted emotions
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# Create a DataFrame for the audio file paths
path_df = pd.DataFrame(file_path, columns=['Path'])

# Combine the two DataFrames into one
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# Map numeric emotion codes to actual emotion labels
Ravdess_df.Emotions.replace(
    {
        1: 'neutral',
        2: 'calm',
        3: 'happy',
        4: 'sad',
        5: 'angry',
        6: 'fear',
        7: 'disgust',
        8: 'surprise'
    },
    inplace=True
)

# Display the first few rows of the processed DataFrame
Ravdess_df.head()

###     2. CREMA-D

In [ ]:
# Get the list of all audio files in the CREMA-D dataset directory
crema_directory_list = os.listdir(Crema)

file_emotion = []  # List to store emotion labels
file_path = []  # List to store file paths

for file in crema_directory_list:
    # Append the full file path to the list
    file_path.append(Crema + file)

    # Extract emotion code from the file name (e.g., '1001_DFA_ANG_XX.wav')
    part = file.split('_')

    # Map the emotion code to the corresponding emotion label
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')  # Fallback in case of unexpected code

# Create a DataFrame for the extracted emotions
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# Create a DataFrame for the corresponding file paths
path_df = pd.DataFrame(file_path, columns=['Path'])

# Combine emotion and path into one DataFrame
Crema_df = pd.concat([emotion_df, path_df], axis=1)

# Display the first few rows of the DataFrame
Crema_df.head()

###     3. TESS

In [ ]:
# Get the list of speaker folders in the TESS dataset directory
tess_directory_list = os.listdir(Tess)

file_emotion = []  # List to store emotion labels
file_path = []  # List to store full file paths

# Iterate through each speaker's folder
for dir in tess_directory_list:
    directories = os.listdir(Tess + dir)  # Get list of audio files in the speaker folder
    for file in directories:
        # Extract the emotion from the file name (e.g., 'OAF_angry.wav')
        part = file.split('.')[0]
        part = part.split('_')[2]  # The third segment contains the emotion label

        # Handle special case: 'ps' stands for 'pleasant surprise'
        if part == 'ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)

        # Append the full file path
        file_path.append(Tess + dir + '/' + file)

# Create DataFrame for extracted emotions
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# Create DataFrame for corresponding file paths
path_df = pd.DataFrame(file_path, columns=['Path'])

# Combine both into a single DataFrame
Tess_df = pd.concat([emotion_df, path_df], axis=1)

# Preview the result
Tess_df.head()

###     4. SAVEE

In [ ]:
# Get the list of all audio files in the SAVEE dataset directory
savee_directory_list = os.listdir(Savee)

file_emotion = []  # List to store emotion labels
file_path = []  # List to store full file paths

# Iterate through each file in the directory
for file in savee_directory_list:
    file_path.append(Savee + file)  # Construct full file path

    # Extract the emotion code from the filename
    part = file.split('_')[1]
    ele = part[:-6]  # Remove the numerical part and extension

    # Map short emotion codes to full labels
    if ele == 'a':
        file_emotion.append('angry')
    elif ele == 'd':
        file_emotion.append('disgust')
    elif ele == 'f':
        file_emotion.append('fear')
    elif ele == 'h':
        file_emotion.append('happy')
    elif ele == 'n':
        file_emotion.append('neutral')
    elif ele == 'sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')

# Create a DataFrame for the extracted emotions
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# Create a DataFrame for the corresponding file paths
path_df = pd.DataFrame(file_path, columns=['Path'])

# Combine the two DataFrames into one
Savee_df = pd.concat([emotion_df, path_df], axis=1)

# Preview the result
Savee_df.head()

### => Kết hợp các tập dữ liệu đã tải

In [ ]:
# Combine datasets from all four sources (RAVDESS, CREMA-D, TESS, SAVEE)
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis=0)

# Xoá trộn tránh bias
data_path = data_path.sample(frac=1).reset_index(drop=True)

# Save the combined DataFrame to a CSV file for future use
data_path.to_csv("data_path.csv", index=False)

# Preview the first few rows of the combined dataset
data_path.head()

## 📊Biểu đồ số lượng mẫu cho mỗi cảm xúc

In [ ]:
# Plotting the number of samples for each emotion

plt.figure(figsize=(10, 6))  # Set figure size for better readability

# Create a count plot for the 'Emotions' column
sns.countplot(x='Emotions', data=data_path, palette='muted')

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Set plot titles and labels
plt.title('Count of Emotions', fontsize=16)
plt.xlabel('Emotions', fontsize=12)
plt.ylabel('Count', fontsize=12)

# Remove top and right borders for a cleaner look
sns.despine(top=True, right=True, left=False, bottom=False)

# Automatically adjust subplot parameters to fit the plot
plt.tight_layout()

# Show the final plot
plt.show()

## III. Khám phá và trực quan dữ liệu - (Exploratory Data Analysis - EDA):
##### 1. Biểu đồ dạng sóng (waveform).
##### 2. Quang phổ âm thanh (spectrogram) - biểu diễn tín hiệu âm thanh trong miền tần số theo thời gian.

In [ ]:
# Function to create a waveplot for a given emotion and audio data
def create_waveplot(data, sr, e):
    plt.figure(figsize=(10, 3))  # Set figure size
    plt.title(f'Waveplot for audio with {e} emotion', size=15)  # Title of the plot
    plt.plot(np.linspace(0, len(data) / sr, num=len(data)), data, alpha=0.7)  # Plot audio data over time
    plt.xlabel("Time (s)")  # X-axis label
    plt.ylabel("Amplitude")  # Y-axis label
    plt.tight_layout()  # Adjust layout for better fit
    plt.show()  # Display the plot


# Function to create a spectrogram for a given emotion and audio data
def create_spectrogram(data, sr, e):
    # Compute the Short-Time Fourier Transform (STFT)
    X = librosa.stft(data)
    # Convert amplitude to decibels
    Xdb = librosa.amplitude_to_db(np.abs(X))
    plt.figure(figsize=(12, 3))  # Set figure size
    plt.title(f'Spectrogram for audio with {e} emotion', size=15)  # Title of the plot
    # Display the spectrogram with the amplitude in decibels
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz', cmap='magma')
    plt.colorbar(format="%+2.0f dB")  # Add color bar
    plt.tight_layout()  # Adjust layout for better fit
    plt.show()  # Display the plot

In [ ]:
def visualize_emotion(emotion, data_path):
    try:
        # Lấy file đầu tiên có emotion tương ứng
        path = np.array(data_path.Path[data_path.Emotions == emotion])[0]

        # Load audio
        data, sampling_rate = librosa.load(path, sr=None)

        # Vẽ sóng âm thanh
        create_waveplot(data, sampling_rate, emotion)

        # Vẽ spectrogram
        create_spectrogram(data, sampling_rate, emotion)

        # Phát audio trong notebook
        display(Audio(path))

    except IndexError:
        print(f"No audio files found for emotion: {emotion}")

In [ ]:
visualize_emotion('happy', data_path)

visualize_emotion('sad', data_path)

visualize_emotion('fear', data_path)

visualize_emotion('angry', data_path)

## IV. Mở rộng dữ liêu - Data Augmentation

In [ ]:
# Data Augmentation functions

def noise(data):
    # Generate random noise amplitude based on the maximum value of the data
    noise_amp = 0.035 * np.random.uniform() * np.amax(data)  # Random amplitude of noise
    # Add Gaussian noise to the original signal
    data = data + noise_amp * np.random.normal(size=data.shape[0])  # Apply noise to the audio data
    return data


def stretch(data, rate=0.8):
    # Ensure that data is a numpy array and of type float32 for time-stretching
    return librosa.effects.time_stretch(np.array(data).astype(np.float32),
                                        rate=rate)  # Stretch the audio signal in time


def shift(data):
    # Generate a random shift range for shifting the audio signal in time
    shift_range = int(np.random.uniform(low=-5, high=5) * 1000)  # Random shift in milliseconds
    # Shift the audio data by the computed range using np.roll
    return np.roll(data, shift_range)  # Shift the audio data in time


def pitch(data, sampling_rate, pitch_factor=0.7):
    # Ensure that data is a numpy array and of type float32 for pitch shifting
    data = np.array(data)
    # Apply pitch shift by a factor (in steps), which will change the pitch of the audio signal
    return librosa.effects.pitch_shift(data.astype(np.float32), sr=sampling_rate, n_steps=pitch_factor)


# Taking any example from the dataset and testing the augmentation functions
path = np.array(data_path.Path)[1]  # Example audio file path
data, sample_rate = librosa.load(path, sr=None)  # Load the audio file and obtain its data and sample rate

#### 1. Ví dụ đơn giản trực quan hoá âm thanh theo dạng sóng
-> Kiểm tra và đánh giá chất lượng dữ liệu đầu vào.

In [ ]:
# Simple audio waveform visualization

# Create a figure with a specified size
plt.figure(figsize=(14, 4))

# Plot the audio signal data as a waveform.
# `np.linspace` is used to create time values based on the number of samples and the sample rate.
plt.plot(np.linspace(0, len(data) / sample_rate, num=len(data)), data, alpha=0.7)

# Set the title of the plot
plt.title("Waveform")

# Label the x-axis as "Time (s)"
plt.xlabel("Time (s)")

# Label the y-axis as "Amplitude"
plt.ylabel("Amplitude")

# Automatically adjust the layout to fit everything properly
plt.tight_layout()

# Display the plot
plt.show()

# Play the audio file using IPython's Audio class
Audio(path)

#### 2. Noise Injection Visualization (thêm nhiễu vào tín hiệu gốc).

In [ ]:
# Noise Injection: Adding random noise to the audio signal

# Inject noise into the original audio data
x = noise(data)

# Create a figure for plotting the noisy signal
plt.figure(figsize=(14, 4))

# Plot the noisy signal as a waveform. `np.linspace` is used to create time values.
plt.plot(np.linspace(0, len(x) / sample_rate, num=len(x)), x, alpha=0.7)

# Set the title of the plot
plt.title("Noise Injection")

# Label the x-axis as "Time (s)" to indicate time duration
plt.xlabel("Time (s)")

# Label the y-axis as "Amplitude" to indicate the amplitude of the audio signal
plt.ylabel("Amplitude")

# Adjust the layout to avoid overlapping labels
plt.tight_layout()

# Display the plot
plt.show()

# Play the noisy audio using IPython's Audio class
Audio(x, rate=sample_rate)

#### 3. Time Stretching Visualization - Thay đổi tốc độ phát âm mà không làm thay đổi cao độ (pitch).

In [ ]:
# Time Stretching: Changing the speed of the audio signal without altering its pitch

# Apply time-stretching to the original audio data
x = stretch(data)

# Create a figure for plotting the stretched signal
plt.figure(figsize=(14, 4))

# Plot the stretched signal as a waveform. `np.linspace` is used to create time values.
plt.plot(np.linspace(0, len(x) / sample_rate, num=len(x)), x, alpha=0.7)

# Set the title of the plot to indicate it's for time-stretching
plt.title("Time Stretching")

# Label the x-axis as "Time (s)" to indicate time duration
plt.xlabel("Time (s)")

# Label the y-axis as "Amplitude" to indicate the amplitude of the audio signal
plt.ylabel("Amplitude")

# Adjust the layout to avoid overlapping labels
plt.tight_layout()

# Display the plot
plt.show()

# Play the time-stretched audio using IPython's Audio class
Audio(x, rate=sample_rate)

#### 4.Time Shifting Visualization – Dịch chuyển tín hiệu âm thanh theo thời gian.

In [ ]:
# Time Shifting: Shifting the audio signal in time (delays or advances the signal)

# Apply time-shifting to the original audio data
x = shift(data)

# Create a figure for plotting the shifted signal
plt.figure(figsize=(14, 4))

# Plot the shifted signal as a waveform. `np.linspace` is used to create time values.
plt.plot(np.linspace(0, len(x) / sample_rate, num=len(x)), x, alpha=0.7)

# Set the title of the plot to indicate it's for time-shifting
plt.title("Time Shifting")

# Label the x-axis as "Time (s)" to indicate time duration
plt.xlabel("Time (s)")

# Label the y-axis as "Amplitude" to indicate the amplitude of the audio signal
plt.ylabel("Amplitude")

# Adjust the layout to avoid overlapping labels
plt.tight_layout()

# Display the plot
plt.show()

# Play the time-shifted audio using IPython's Audio class
Audio(x, rate=sample_rate)

#### 5. Pitch Shifting Visualization — Thay đổi cao độ (pitch) của tín hiệu âm thanh mà không thay đổi tốc độ phát.

In [ ]:
# Pitch Shifting: Changing the pitch (frequency) of the audio signal

# Apply pitch-shifting to the original audio data
x = pitch(data, sample_rate)

# Create a figure for plotting the pitch-shifted signal
plt.figure(figsize=(14, 4))

# Plot the pitch-shifted signal as a waveform. `np.linspace` is used to create time values.
plt.plot(np.linspace(0, len(x) / sample_rate, num=len(x)), x, alpha=0.7)

# Set the title of the plot to indicate it's for pitch-shifting
plt.title("Pitch Shifting")

# Label the x-axis as "Time (s)" to indicate time duration
plt.xlabel("Time (s)")

# Label the y-axis as "Amplitude" to indicate the amplitude of the audio signal
plt.ylabel("Amplitude")

# Adjust the layout to avoid overlapping labels
plt.tight_layout()

# Display the plot
plt.show()

# Play the pitch-shifted audio using IPython's Audio class
Audio(x, rate=sample_rate)

## V. Trích xuất đặc trưng - Feature Extraction

In [ ]:
# Feature Extraction (updated for newer APIs)
def extract_features(data):
    """
    This function extracts various audio features from the given audio data.
    Features include zero-crossing rate (ZCR), chroma_stft, MFCC, RMS, and mel spectrogram.
    """
    result = np.array([])  # Initialize an empty array to store features

    # Zero-Crossing Rate (ZCR): Measure of how many times the signal changes sign
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result = np.hstack((result, zcr))  # Append the ZCR feature to result array

    # Chroma Short-Time Fourier Transform (chroma_stft): Measures harmonic and melodic content
    stft = np.abs(librosa.stft(data))  # Short-time Fourier transform of the audio signal
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)  # Compute chroma features
    result = np.hstack((result, chroma_stft))  # Append the chroma feature to result array

    # Mel-frequency Cepstral Coefficients (MFCC): Represent the short-term power spectrum of sound
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=13).T, axis=0)  # Extract 13 MFCCs
    result = np.hstack((result, mfcc))  # Append the MFCC feature to result array

    # Root Mean Square (RMS): Energy of the audio signal
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)  # Calculate RMS value
    result = np.hstack((result, rms))  # Append the RMS feature to result array

    # Mel Spectrogram: Spectrogram representation of the audio signal using Mel scale
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)  # Extract Mel spectrogram
    result = np.hstack((result, mel))  # Append the Mel spectrogram feature to result array

    return result  # Return the array of extracted features


def get_features(path):
    """
    This function loads the audio data from the given path, extracts features from the original data,
    and applies data augmentation techniques (noise, stretching, and pitch shifting) to generate additional features.
    """
    # Load the audio file (duration and offset are specified to load a portion of the audio)
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)

    # Base features: Extract features from the original audio data
    res1 = extract_features(data)
    result = np.array(res1)  # Store base features as the result

    # Noise Augmentation: Apply noise injection and extract features from the noisy audio
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2))  # Append noisy features to the result

    # Stretching and Pitch Augmentation: Apply time-stretching and pitch-shifting, then extract features
    new_data = stretch(data)  # Apply time-stretching
    data_stretch_pitch = pitch(new_data, sample_rate)  # Apply pitch shifting
    res3 = extract_features(data_stretch_pitch)  # Extract features from the stretched and pitch-shifted data
    result = np.vstack((result, res3))  # Append augmented features to the result

    return result  # Return the final feature set as a 2D array

##### Trích xuất đặc trưng cho toàn bộ dataset âm thanh kèm theo nhãn cảm xúc tương ứng

In [ ]:
# Extract features for the entire dataset
X, Y = [], []  # Initialize lists to hold features (X) and labels (Y)

# Loop through each audio file and its associated emotion
for path, emotion in zip(data_path.Path, data_path.Emotions):

    # Extract features for the current audio file (including augmented variants)
    features = get_features(path)  # Extract features for the original audio and its augmented versions

    # Loop through the extracted features (original, noisy, stretched + pitched) and append them to X
    for feature in features:
        X.append(feature)  # Append the feature to the feature list (X)
        Y.append(emotion)  # Append the corresponding emotion label to the label list (Y)

        # Repeat 3 times because we have 3 variants (original, noise, and stretch + pitch)
        # This ensures that for each audio file, we have 3 corresponding entries in X and Y

##### Kiểm tra số: Mẫu đặc trưng (X) và số nhãn tương ứng với các mẫu (Y)

In [ ]:
# Check the length of the feature set (X), the labels (Y), and the number of paths in data_path
len(X), len(Y), data_path.Path.shape

##### Lưu trữ dữ liệu đặc trưng và nhãn của toàn bộ dataset

In [ ]:
# Save the features and labels into a DataFrame
Features = pd.DataFrame(X)
Features['labels'] = Y

# Save to a CSV file
Features.to_csv('features.csv', index=False)

# Display the first few rows
Features.head()

##### Chuẩn bị dữ liệu đặc trưng và nhãn

In [ ]:
# Features
X = Features.iloc[:, :-1].values  # Extract all columns except the last (labels)

# Labels
Y = Features['labels'].values  # Extract the labels column

##### Mã hóa nhãn dạng phân loại thành dạng vector one-hot (chuẩn hóa cho bài toán phân loại đa lớp).

In [ ]:
# One-hot Encoding for labels
# As this is a multiclass classification problem, we are one-hot encoding the labels (Y).
encoder = OneHotEncoder(sparse_output=False)

# Convert labels (Y) to a 2D array (required by OneHotEncoder) and apply fit_transform.
# This will convert each categorical label into a binary vector (one-hot encoded).
Y = encoder.fit_transform(np.array(Y).reshape(-1, 1))

##### Chia dữ liệu thành tập huấn luyện và tập kiểm tra

In [ ]:
# Splitting data into training and testing sets
# We use train_test_split from scikit-learn to split the data (X) and labels (Y)
# into training and testing sets. The random_state ensures reproducibility of the split.

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)

# Checking the shape of the train and test sets
# This will output the number of samples and features in both training and test datasets
x_train.shape, y_train.shape, x_test.shape, y_test.shape

##### Thực hiện chuẩn hóa (standardization) đặc trưng (features) trong tập huấn luyện và tập kiểm tra:

In [ ]:
# Feature scaling (Standardization)
# We are using StandardScaler from sklearn to standardize the features (X).
# This ensures that each feature has a mean of 0 and a standard deviation of 1.
# The scaler is first fitted on the training data and then applied to both training and testing data.

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # Fit the scaler to the training data and transform it
x_test = scaler.transform(x_test)  # Apply the same transformation to the testing data

# Checking the shape of the scaled data
# This will output the number of samples and features in both the scaled training and testing datasets
x_train.shape, y_train.shape, x_test.shape, y_test.shape

## VI. Xây dựng mô hình

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Logistic Regression không hỗ trợ đầu ra one-hot trực tiếp,
# nên ta dùng OneVsRestClassifier để xử lý phân lớp đa nhãn
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='lbfgs'))

## VII. Huấn luyện mô hình

In [ ]:
model.fit(x_train, y_train)

## VIII. Đánh giá mô hình
#### Đánh giá mô hình trên tập kiểm tra và Vẽ đồ thị thể hiện quá trình học

In [ ]:
# Make predictions on the test set
# The 'predict' method returns the predicted probabilities for each class.
pred_test = model.predict(x_test)

# Convert the predicted probabilities to the actual class labels using the encoder
# 'inverse_transform' converts one-hot encoded predictions back to original labels
y_pred = encoder.inverse_transform(pred_test)

# Convert the actual labels (y_test) from one-hot encoding to original labels for comparison
y_test_labels = encoder.inverse_transform(y_test)

In [ ]:
from sklearn.metrics import accuracy_score

# Tính độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of our Logistic Regression model on test data: {:.2f}%".format(accuracy * 100))


##### So sánh trực quan nhãn dự đoán của mô hình (Predicted Labels) với nhãn thật của dữ liệu test (Actual Labels).

In [ ]:
# Create a DataFrame to compare predicted labels with actual labels
# 'y_pred' contains the predicted labels and 'y_test_labels' contains the true labels

df = pd.DataFrame({
    'Predicted Labels': y_pred.flatten(),  # Flattening in case it's a multi-dimensional array
    'Actual Labels': y_test_labels.flatten()  # Flattening to match the structure of 'Predicted Labels'
})

# Display the first 10 rows of the DataFrame to compare predictions with actual labels
df.head(10)

#### Ma trận nhầm lẫn (confusion matrix)

In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_test_labels, y_pred)

# Retrieve the list of labels from the encoder
labels = encoder.categories_[0]  # This will give the emotions in the same order as the encoder

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(pd.DataFrame(cm, index=labels, columns=labels),
            linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='d')

# Add titles and labels to the plot
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)

# Display the plot
plt.show()

#### Báo cáo phân loại (classification report)

In [ ]:
# Generate and print the classification report
print(classification_report(y_test_labels, y_pred))

## IX. Xuất mô hình

In [ ]:
import joblib

joblib.dump(model, "emotion_model.joblib")